GA Capstone Project Technical Report
==
Predicting NBA potential
--
**Matt Lee**

----

Problem Statement
--
Can we predict how successful a player will be in the NBA based on data alone? 

Basketball analytics have been a growing trend, more models and rankings are available online, for example, 538's CARMELO model. However, every year around late June, the NBA Draft arrives, and the teams just never get it right. Top picks are wasted as hyped players turn into busts, while late sleepers way down in the draft turn into blossoming superstars (e.g. Isaiah Thomas, picked last in the draft). 

Organizations spend a ton of money to fly scouts around the world to identify potential top picks, and hire top analysts to predict potentially successful NBA players. When the team lands someone like Michael Jordan, the benefits are enormous. The team increases its chances to make a championship run, ticket and merchandise sales increase, and the local economy gets a boost. Unfortunately, this is not always the case, and many draft picks were wasted this way (and why so many draft picks are included in trades). 

My goal is to make a model that can accurately predict the outcome of college players in the NBA, in turn saving costs and enabling teams to know what they are getting in return. 

Data Acquisition
--
My initial goal was to incorporate basketball statistics, such as Points per Game, demographic statistics, such as the poverty level of where the player grew up, how many siblings the player had, and physical measurements, such as hand size, vertical reach. This would enable our model to have a unique and more complete understanding of each player, and hopefully produce more accurate results than other models out there. 

Since demographic statistics are extremely difficult to compile (e.g. manually entering data from Wikipedia), I am initially approaching the problem with just basketball statistics and physical measurements. I ambitiously gathered nine sets of data from credible sources like BasketballReference, DraftExpress, RealGM, and the official site of the NBA with the goal of using basketball and physical measurement data before a player's career in the NBA to predict their NBA-level statistics. 

This lead to many problems when attempting to compile the data together, for example:
* Different sources spelling player names differently
* Advanced physical measurements such as hand size, vertical reach, wingspan was not recorded before the year 2000
* Players do not have to attend the Draft Combine, where all measurements are recorded
* College basketball statistics are available only from 2003 onwards
* International players and players who entered the NBA from highschool are excluded
* Not all drafted players play in the NBA, and some players leave after a very short term
* Players do not stay in college for a fixed number of years

Data Organization
--
I scraped and downloaded data by year, and compiled them into a single dataframe and adding a 'Year' column to differentiate them through a custom function. 

I decided to make a foundation dataframe (with names based off of BasketballReference) that had columns that were generally available and complete so they could serve as primary keys when I join features from other datasets. These features included:
* Year
* Draft Position
* Name
* NBA Team
* College

This was somewhat helpful, but I still had to manually edit some of the names from other sources. I also added the player's birthdate, birth city and state. 

To compile college basketball statistics, I scraped a total of 65,535 rows of yearly statistics of players from 2003 to 2017. The way I organized college data was based off of how I operationalized my target variable (see below). I filtered players who were in the NBA and fit the criteria of my operationalized target, and aggregated the college data by taking the average of the statistics throughout the player's college career. To ensure I had aggregated that right player (there were many cases of players sharing the same name but attending different schools), I had to compare and manually edit names and colleges. 

I also transformed data according to their distribution by looking at kernel density estimation plots, reducing skewness and since linear models have been working the best, making it easier to see a linear relationship.

Operationalizing the Target - Timing
--
My goal was to produce an ordered list where players are ranked as accurately as possible according to their skill based on advanced metrics, so that each team at each draft position will know what their return is. 

The stakeholders in this problem are NBA teams, primarily the team owners and GMs; individuals who have a say in the draft and in the structure of the team. From their point of view, being able to draft and sign a rookie player to a long-term extension would be the optimal outcome. This mitigates uncertainty with team structure, and allows for a peace of mind as they know to a degree of certainty what their return is from the draft. So what would be the best information for the stakeholders to have when making this decision?

We have to first take a look at how a NBA rookie contract works. Once a player is drafted, his rookie contract guarantees two years of pay, and salary is based on the position drafted. After the completion of the player's first season until October 31st, the team is allowed to exercise an option to re-sign the player for a third year. This also applies for the following year, to re-sign the player for a fourth year. Teams are also allowed to offer rookie extensions to players if they want to retain him for long-term, however, they must play out the four years of their rookie contract. If no extension is given, after the fourth year, the player becomes a Restricted Free Agent(RFA). This means he can sign a long term contract with his current team, sign an offer with another team and see if the current team matches, or sign a one year qualifying offer with the current team. 

Due to the nature of the rookie contract, it is best for teams to sign extensions as quickly as possible to retain the player and prevent other teams from offering a higher price. 

Predicting the outcome of a newly drafted player's first and second season gives minimal value to the team as the first two years of a rookie contract is guaranteed. Third year statistics is a possible option, as remarkable results may indicate a rising superstar, however, it has room for errors such as a player having a fluke season or the player is a late bloomer. The team also has the leverage to exercise the 3rd year option to keep the player, which is cheaper than offering an extension. This same logic applies for predicting 4th year results. This brings us to year 5, where the team has no control over the player. 5 years is a generous amount of time for a player to flourish and develop his skills, and typically is either approaching or just reached his prime. Due to these reasons, I will be using the average statistics (mitigating effect of outliers) of the first five years of a player's career in the NBA as my target variable. 





Operationalizing the Target - Metrics
--
As I mentioned above, I want to create an ordered list that ranks players according to their skill, so teams will know what they're getting in return for the draft pick. I will be utilizing NBA advanced metrics to measure skill. There has always been an argument on the accuracy and validity of a single advanced metric, as most of them are comprised of box-score stats, which do not capture defense very well. For this reason, I will be using a combination of metrics to generate a single measure that encompasses more factors. 

Additionally, to give teams more choice, I will produce three lists with different models behind each, that ranks players according to overall skill, offensive skill, and defensive skill. 

My approach to combining these metrics was to test out Principal Component Analysis (PCA) and Unity-Based Normalization. 

For measuring overall talent, these metrics were considered:
* PER
* Win Shares
* Box +/-
* Value Over Replacement Player

Offensive metrics:
* Effective Field Goal %
* True Shooting %
* Free Throw Rate
* Usage %
* Offensive Rebound %
* Offensive Win Shares

Defensive metrics:
* Defensive Rebound %
* Defensive Win Shares
* Defensive Box +/-

There isn't a perfect way of knowing which combinations or the method of combining the metrics are the best, so I just tested by trial and error based on model test scores.

Models
--

I first ran default parameter regression models cross-validated with train-test-splits, such as **Linear Regression**, **RidgeCV**, **LassoCV**, **KNeighbors Regressor**, **Decision Tree Regressor**, **Random Forest Regressor**, **Extra Trees Regressor**, **Bagging Decision Trees Regressor**, **Adaboost Regressor**, **Gradient Boost Regressor**, and **Extreme Gradient Boost Regressor** with a selection of input variables that aren't highly correlated with each other (e.g. Defensive Rebounds and Total Rebounds), with the combined metric reflecting each list I wanted to make (Overall, Offense, Defense).

Linear models seemed to generate the best R-squared scores from the hold-out data, with Gradient Boosting Regressor doing the best among the other types of models, thus I chose to focus on **Linear Regression**, **RidgeCV**, and **LassoCV**.

I used **SelectKBest** to incorporate some feature selection techniques, and iterated over the number of features that would give me the best test score. Typically, a higher number of features worked best for all three scenarios. 

**GridSearchCV** was used to further cross-validate Linear Regression models, and I tested out different alpha penalties for Ridge and Lasso models with a default number of stratified k-folds for cross-validation. 

I mainly evaluated models by the R-squared value on the test set, and will consider root mean-squared error as well. Predictions of the defense metric were the most accurate, achieveing test R-squared scores of around 74%. Offense and overall skill predictions were not as accurate, achieving R-squared scores of around 40%. 

However, to analyze the effectiveness of these models, we need to match the predictions with the player and rank them, then compare the results by a specific draft year and see if the sequential picks from our list resulted in teams selecting better than they did. We can also rank players by the combined metrics we created and see how well our results did.  

Further Considerations
--
Obviously, I would like to work with more data that are not basketball related, and hopefully cover some of the intangible qualities of successful players through additional features. 

I am also working with a pretty small dataset as a result of my data cleaning and aggregation process. Further polishing this process and attempting to include more players would help the performance of the models. 

Since I am focusing on three linear models, I would like to utilize some ensemble methods to incorporate their predictions together and see if it produces a better result. Averaging the predictions is one method, as well as using a model trained on these predictions. 

Lastly, I would like to be able to compare all the possible drafts that the data I have can cover and see how many teams could have improved their draft picks, as well as inputting a new batch of incoming players every year before the NBA Draft and compare the players picked by my model and by actual NBA teams. 